In [52]:
import pandas as pd
import numpy as np
import os
import requests
import re

In [37]:
import warnings
warnings.filterwarnings('ignore')

# Concatenate monthly-end Billboard Hot 100

In [38]:
path = 'billboard_data/'
file = os.listdir(path)

year = [i for i in range(2012, 2022)]

file_name = sorted([f for f in file if f[:4] in str(year)])

hit = pd.read_csv(path + file_name[0])

for f in file_name[1:]:
    hit = pd.concat([hit,
                     pd.read_csv(path + f)],
                     ignore_index=True)
    
# add chart date for further reference 
## (i.e. if we want to denote songs ever featured top 20 as hit, chart date matters)
date = []

for i in file_name:
    date += [i[:-4]] *100
    
hit['date'] = date

hit

,artist,isNew,last,peak,rank,title,week,date
0,Adele,False,2,1,1,Set Fire To The Rain,21,2012-02-04
1,Rihanna Featuring Calvin Harris,False,1,1,2,We Found Love,18,2012-02-04
2,Flo Rida,False,3,3,3,Good Feeling,17,2012-02-04
3,David Guetta Featuring Nicki Minaj,False,10,4,4,Turn Me On,8,2012-02-04
4,Katy Perry,False,6,3,5,The One That Got Away,15,2012-02-04
...,...,...,...,...,...,...,...,...
11695,Kenny Chesney,False,100,87,96,Knowing You,6,2021-10-02
11696,Lil Nas X Featuring Miley Cyrus,True,0,97,97,Am I Dreaming,1,2021-10-02
11697,Zac Brown Band,True,0,98,98,Same Boat,1,2021-10-02
11698,H.E.R. Featuring Chris Brown,False,95,64,99,Come Through,15,2021-10-02


In [39]:
hit_unique = hit.drop_duplicates(subset=['title','artist'])[['title', 'artist','date']]
hit_unique['hit'] = [1]*len(hit_unique)
hit_unique.reset_index(inplace=True, drop=True)
hit_unique

,title,artist,date,hit
0,Set Fire To The Rain,Adele,2012-02-04,1
1,We Found Love,Rihanna Featuring Calvin Harris,2012-02-04,1
2,Good Feeling,Flo Rida,2012-02-04,1
3,Turn Me On,David Guetta Featuring Nicki Minaj,2012-02-04,1
4,The One That Got Away,Katy Perry,2012-02-04,1
...,...,...,...,...
3267,7am On Bridle Path,Drake,2021-10-02,1
3268,Off The Grid,Kanye West,2021-10-02,1
3269,In Da Getto,J Balvin & Skrillex,2021-10-02,1
3270,Am I Dreaming,Lil Nas X Featuring Miley Cyrus,2021-10-02,1


In [40]:
hit_unique['track_id'] = [np.nan] * len(hit_unique)
hit_unique['release_date'] = [np.nan] * len(hit_unique)
hit_unique['spotify.name'] = [np.nan] * len(hit_unique)
hit_unique['spotify.artists'] = [np.nan] * len(hit_unique)

## Get Spotify track ID

In [41]:
def getArtistNameList(result_artists):
    artists = []
    for a in result_artists:
        artists.append(a['name'])
    return artists

### Direect search by title and artists

In [50]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

try:
    start_idx = hit_unique.loc[~hit_unique['track_id'].isnull(),:].index.values[-1] + 1
except:
    start_idx = 0
    
for i in range(start_idx, len(hit_unique)):
    # direct search by title and artist
    title = hit_unique['title'][i]
    artist = hit_unique['artist'][i]
    query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                        artist,
                                                                        hit_unique['date'][i][:4])


    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

    try:
        results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=True)

        for j in range(len(results)):
            artists_i = getArtistNameList(results['artists'][j])

            for a in artists_i:
                # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'][j])
                    hit_unique['track_id'][i] = results['id'][j]
                    hit_unique['release_date'][i] = results['album.release_date'][j]
                    hit_unique['spotify.name'][i] = results['name'][j]
                    hit_unique['spotify.artists'][i] = artists_i
                    break
            break
    except:
        pass

2606 41L3O37CECZt3N7ziG2z7l
2607 4wNIkl5XGiAACjFBlDWuSd
2608 1DWZUa5Mzf2BwzpHtgbHPY
2610 7lidXGPXPYLNThITAOTlkK
2611 5SiZJoLXp3WOl3J4C8IK0d
2613 2u3EyuCqqV31woqGvO9LBz
2615 2hwOoMtWPtTSSn6WHV7Vp5
2616 4jXl6VtkFFKIt3ycUQc5LT
2618 24NuEDgyRIljwSyY0lsr7b
2621 3WfSNAoj2D8C9noPSGZZFM
2622 2Yv2mHzr5AQavVdwQjEokV
2623 7ccTcabbJlCJiIqtrSSwBk
2624 3R8CyhJfVjvgIROd5RSGhQ
2625 7Dj2wqPYXfSRFwa6qyOztr
2627 3Qa944OTMZkg8DHjET8JQv
2628 601fZrsuXVF0yxhCTaEKZf
2629 438wODjSTL2jgn1nePaNew
2630 1OuN92HcVG6NVpWbeESNB3
2631 7HMmFQsKsljwTw8bS7lu19
2633 34d7sGX6WUqq8f04e7CSNF
2636 1ow7j1d2ZGFNmSpwmM3qNO
2637 3fATn2bGmTGvw49p5yw4qg
2640 1uCVlCp60dEJBsPp6033zj
2641 1kwnxJNVl7cwcU98RvMBaR
2643 3QVHSV8YKYq4L8tI5rnFgj
2644 70bbnfAQrEe77yDsombX1j
2645 0udOCZp7yk0Ml92C0k8H8D
2647 73SpzrcaHk0RQPFP73vqVR
2650 2ZTYlnhhV1UAReg7wIGolx
2651 6ZelF5APDN5r6XnFqcnvWR
2652 6mpYFu4VudWKLheOaWYHng
2653 22LAwLoDA5b4AaGSkg6bKW
2654 1aNaUNjSp4hoiKwahgWhOu
2658 3hjmowsZzUSY1xY2XQVyIy
2659 6PYDnaVgRY9PedhbNEDQXN
2661 27829SWtwhWUANV

3048 472vIK1ldetTxRxG3ovaiY
3051 5hChuUMe5iHfjkXgzj1D4b
3052 1RohgA8t3qiVc5HcAroDgf
3055 3aQem4jVGdhtg116TmJnHz
3057 4u4NyuceXP7Uzh7XFJKCr1
3058 3ybREoYkNekNCjGBsJ8va9
3060 5fT8wGwWujRP921VP6Gp8f
3064 3T03rPwlL8NVk1yIaxeD8U
3065 6I3mqTwhRpn34SLVafSH7G
3066 0oaY19dUwZimIgzn3ZZLZO
3068 5NU40QTlXrDUJzDBdv79bg
3071 1m4bwZNQYpDHAfLgklcTjP
3074 4BJv3G0tWwI1mC3dNcUFTL
3075 2Z0Ie1NJJLkp14ClmTx5P5
3076 00KyYtT6NaXwbPecina5Pj
3077 3qiFXINhnsIAkaE5AKPiKO
3078 7bnniUFU8K59U2qBzX6yxO
3079 32L05FkUrYfVlc0IwJ45ld
3082 6b4hBdXxzHMWqL5XteNgtJ
3084 00JscSANWnT44OwnwADlwo
3088 4mNigCKsNSfLqMc18KKT8Z
3089 4Bd004ggJHbiABzWBzyQah
3092 7FdUvDkaE24o3FPIWTvzv2
3093 3XrRaLjae1DKUtt2KvNFr1
3094 0NmuYnjETG3u3qx0OmEJev
3095 7l2tmgUhV7Y2aJHjiszifg
3096 1ZIKpMTRPqhwAoVXWY3tXI
3098 4eI3W6rWDbgrJyCfWbAOHm
3100 1pxTuhHXNfsJefQiTY8V5D
3101 1mAey8kEPmDM2icL56kS9d
3102 2bgTY4UwhfBYhGT4HUYStN
3104 5CZ40GBx1sQ9agT82CLQCT
3105 6SRsiMl7w1USE4mFqrOhHC
3106 2TOzTqQXNmR2zDJXihjZ2e
3107 2tGvwE8GcFKwNdAXMnlbfl
3108 5JCoSi02qi3jJeH

### Search by title with artist traversal

In [53]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in hit_unique.loc[hit_unique['track_id'].isnull(),:].index.values:
    title = hit_unique['title'][i]
    # clean title
    #title = ''.join(c for c in hit_unique['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in hit_unique['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | Featuring ', hit_unique['artist'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                            artist,
                                                                            hit_unique['date'][i][:4])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=True)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        hit_unique['track_id'][i] = results['id'][j]
                        hit_unique['release_date'][i] = results['album.release_date'][j]
                        hit_unique['spotify.name'][i] = results['name'][j]
                        hit_unique['spotify.artists'][i] = artists_i
                        break
                break
            break
        except:
            pass

1 2CvK6oqk5P4kHenbt2nFVT
3 4zTnEW54zezqmWyeGibRGV
15 3xs1uYjjw07JYxqLtsbq95
20 1kPrEf0qjfxOLP05kaXgkb
23 25BU1YwIUmu6hiih7tLNUN
28 5nlMlqfbxJxX94DJrcRy6Y
49 2llbj6W90VcwPBSnfQXLrt
50 0gFkruQrXlxIOo738m76Xb
51 2NRUU0gP1d8Yif8CpsCW8l
58 0yQw3d0iQDjMrxaVHFv77J
91 5ORf8BZMbq4xN9kiumrPcQ
93 7drd8T07dm62NkcYESlVhu
97 1WG5uCUCD8j11b48YA1dKD
107 2woCC5hiuPTDAonxhEojP0
108 02j4EgmynGX4k3uC18INk1
111 19TPvVmCUHn71omJ16N9hK
119 17cEIUjCcycYnE9sT8l9TK
131 0QMdQUE5jtL9jih2ZvDJL3
134 7lUhtLCdqomO7jsZFkVcBW
139 7KfqzvwFm0LFX7O3yOopJq
147 1leMorBNkPWVehL4pIqkOs
149 3hTfNt05rYtyMamwkxyHaZ
157 1LmN9SSHISbtp9LoaR5ZVJ
160 4Qj378pBslTXFC3o4DFWRJ
165 0nq6sfr8z1R5KJ4XUk396e
167 4It00tEYLwpjQtAhqnjCPC
169 2bgW1SJqUjIkWcbxTPTIdy
173 1fI5yYMJEYkAHI5YRdnikY
176 2lD4LsB3iQVnddkVM2RFAl
180 5aTt7sopAOdIECnddgnp5B
186 33tORbMH4bNhiDVKUGbqEF
189 17zF2ISrDrQdaULA8cxReK
197 6j9iyrrmqWlQZ5SD1hSTaq
203 0k3yajjoN6uZINmPe2uKrt
205 49mzTLlFgVYtZUE4J0UuMU
207 5DB3QBbVXcnpL38eLSp6Gp
211 75lo8VozcCLRG9WGG674iD
213 4MevEW6tsWrp

1538 41ZWQ6xczwnzsav1nlDXrJ
1539 2PC9Tl8SWVO2yhDDtJcrAi
1544 2q95XoeFGixx8b5LNF6Ey1
1557 5GXAXm5YOmYT0kL5jHvYBt
1571 2Eomvb6YHo8lhfleLufuca
1572 3a1lNhkSLSkpJE4MSHpDu9
1575 5knuzwU65gJK7IF5yJsuaW
1583 1r4pDJCz485hOhBxIXHDPV
1585 3VCYe6mCXebBgcfzaxCVNI
1586 3u54uJdewwAvXMjOtHZZKs
1589 0PUWNk7gooEAHUMW5V6e39
1594 5meCB8rEqSyiRRBED3Fl7e
1595 3J7J46rmHmYw90NVqVvXU4
1606 06ObEzCPd14uMs7hNBFFCJ
1611 36DHSYPiIIf1Pl0irqyebq
1615 1KzF176eH3k51UwhuQHgr3
1618 04gpBAQIjpgdUDOa1Et0c4
1627 4ntxvAQ2V4ofKQB9XNVHqK
1629 2tMByQa6qGxHc8Ccqtxpsm
1636 6Uf8poA3APtzQiJrrqomWO
1645 6gpcs5eMhJwax4mIfKDYQk
1646 6oJwZFH3K6KoUXltDqTpGu
1650 32bqxZEJRJObeNyc8bEiPt
1661 0DSSnLy9UYNdiOMuQclRrk
1665 4RIl218uURN8ba6Xd6Te4n
1667 5CRHxhQOzuj5iPASXodlmm
1671 5EBr6I5VPfut2oUyyR7b5E
1677 6kex4EBAj0WHXDKZMEJaaF
1680 0jy1pezjONZf68MaI0Yp2Z
1681 2bjwRfXMk4uRgOD9IBYl9h
1684 1f5cbQtDrykjarZVrShaDI
1687 4ut5GTELyIaTLJmkTPpRws
1690 7BKUWh5sdtdS17crohRe4L
1691 1XtsVai7ihn3nVWozzuEfy
1697 77lGafvlU68CeHgB2pkHC9
1699 2d48mamATjby7wg

2878 4pvb0WLRcMtbPGmtejJJ6y
2880 5u1n1kITHCxxp8twBcZxWy
2885 3zxAw8PaqpeGZ3Pakkxv3J
2892 7Bar1kLTmsRmH6FCKKMEyU
2896 0wI5Rs6ek3PvBd4tW61X3M
2898 5faePb78WV2AovR1kbrsbR
2900 0pM9RJkkEuGnDsdPDmGgPR
2901 2T5NBwKRySiCR78vVk08vr
2903 33gwZOGJWEZ7dRWPqPxBEZ
2907 2aVrAM2aWQukrVc7yh5wQz
2911 5dchdf3LoipmlZFKYm4xcE
2913 5JHNyK0kYBB7imBqbr7if8
2916 3QGppKBPd9gHOgHJzRbVIw
2925 73P6M7lSnS4heaXSae5RlM
2939 5JqZ3oqF00jkT81foAFvqg
2945 5Bnh2sJPeQCdpoZD4UKzsX
2952 3MEruRteiUZXkStfTlZqRn
2953 5gEN5oFKfPDiwLNaLWbR2o
2955 4h30gjsvh2TrEQpIBvqiac
2984 1yOQAPEAfjqUduXX0DnR3a
2989 6uwfVkaOM1mcMkFmSn35ix
2997 1APAEceXOVOP0LYE0C4NBG
2998 2etHQJxIbV0soyPhelVs9Y
3005 6toQdWWc4noiOk3Eo5mVDS
3023 3QXEmbGgpXX4cR8VZ662m1
3026 08QAkBRyWLUQ3UniIyMFtP
3027 3R6wLBD4oU6op8L4YfMho9
3037 6SN1H9p6A0UbsNpfkxEEX6
3038 61FfbHfk4qT8ZzNA5dqN9j
3039 6Qrai9GOIMzU6URzM6F7zs
3040 0FrJf87J2nMq5Tk7z6FurB
3050 7cDXETYT08xTvMI1fAnrWt
3053 4iJyoBOLtHqaGxP12qzhQI
3056 65OVbaJR5O1RmwOQx0875b
3059 7AWdaXn8R5zDIL6XMOn4lx
3061 6F2r4HgpJnvKDmk

In [54]:
hit_unique.loc[hit_unique['track_id'].isnull()]

,title,artist,date,hit,track_id,release_date,spotify.name,spotify.artists
5,It Will Rain,Bruno Mars,2012-02-04,1,NaN,NaN,NaN,NaN
6,Sexy And I Know It,LMFAO,2012-02-04,1,NaN,NaN,NaN,NaN
7,Stronger (What Doesn't Kill You),Kelly Clarkson,2012-02-04,1,NaN,NaN,NaN,NaN
8,Ni**as in Paris,Jay Z Kanye West,2012-02-04,1,NaN,NaN,NaN,NaN
11,"Young, Wild & Free",Snoop Dogg & Wiz Khalifa Featuring Bruno Mars,2012-02-04,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3225,I Guess I'm In Love,Clinton Kane,2021-09-04,1,NaN,NaN,NaN,NaN
3230,Knowing You,Kenny Chesney,2021-09-04,1,NaN,NaN,NaN,NaN
3250,Have Mercy,Chloe,2021-10-02,1,NaN,NaN,NaN,NaN
3254,Lonely,DaBaby X Lil Wayne,2021-10-02,1,NaN,NaN,NaN,NaN


### Search by title(cleaned) and traverse artists

In [55]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
    
    
for i in hit_unique.loc[hit_unique['track_id'].isnull(),:].index.values:
    #title = hit_unique['title'][i]
    # clean title
    title = ''.join(c for c in hit_unique['title'][i] if c.isalnum() or c.isspace())
    ##title = ' '.join(c for c in hit_unique['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | Featuring | X ', hit_unique['artist'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                            artist,
                                                                            hit_unique['date'][i][:4])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=True)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        hit_unique['track_id'][i] = results['id'][j]
                        hit_unique['release_date'][i] = results['album.release_date'][j]
                        hit_unique['spotify.name'][i] = results['name'][j]
                        hit_unique['spotify.artists'][i] = artists_i
                        break
                break
            break
        except:
            pass

7 00b09Hp7Q8P9oLaK1PspfX
11 5MGF8hO2Qd44X8LanSDsJX
39 5ZRKAeZJsZFsrDT7SwJfL4
71 05pKAafT85jeeNhZ6kq7HT
90 0z9UVN8VBHJ9HdfYsOuuNf
96 6x0M179l7MO6uK3ye6mupF
226 1YTO55iUyFqx1QgcWXArLB
241 4xJzZkAP33kMkknd8cZBiM
280 043bfUkTydw0xJ5JjOT91w
282 2ikkgfIIM9kGKZCXzXnTpj
331 3Tu7uWBecS6GsLsL8UONKn
337 0NG5gJzPHjYeQmqcmqfopN
342 6MVdz8I7oYm86MRzZiOZa8
343 6J5sxraPPZ4b0CVOGAgpXj
344 2iJuuzV8P9Yz0VSurttIV5
367 4mQVHEjrnuUd7G5IVhSYTk
378 6PQ0WLZnT479J7Q1TGIb3n
405 5sdhqXSfvyDgU6fiDJTP4f
411 4FQDZwcvvXpqpPMquZ6qTV
413 01TuObJVd7owWchVRuQbQw
438 6LoQHIo74tOzQ8EsLEkhgF
459 3kxDlGTiY5oXxrtyLgvxBy
475 5vTPxzm4h2bY9rYyVrGEU5
477 2y4lAQpi5VTNLu2ldeTdUH
480 3Qe097eJNp9MfpT2PieEw0
485 00QyLmjxaSEE8qIZQjBXBj
501 0UAmPUdpTRq8w54B3H8TzY
517 3c7MNgwEBQTnrOWLjU1DHq
537 2aSC2xhRxOLiiZZVjhbylH
568 3BnBKNItMng824sLx0pUfy
600 1jhrjM26NnMt9iSD7pllmP
669 3yNVRe7CmqPYE0RkWWqry6
676 6IDDwI0YOCAUDhMZltQekS
701 2V3EBbauVyLyc8nU2igZyJ
702 4tzrdaXrV7FQMLoM0FvXdZ
709 1HOlb9rdNOmy9b1Fakicjo
712 7EbMEfsCq8VJiKGyCSuPyu
750 5aj0